system:

`$ lsb_release -a` 
`$ uname -a` 
```
Ubuntu 18.04.1 LTS
Linux c220g2-011019 4.15.0-88-generic #88-Ubuntu SMP Tue Feb 11 20:11:34 UTC 2020 x86_64 x86_64 x86_64 
GNU/Linux
```
`$ sudo smartctl -a /dev/sdc` 
```
Model Family:     Intel 730 and DC S35x0/3610/3700 Series SSDs
Device Model:     INTEL SSDSC2BB480G4
Serial Number:    BTWL523503PV480QGN
LU WWN Device Id: 5 5cd2e4 04c04ba01
Firmware Version: D2010374
User Capacity:    480,103,981,056 bytes [480 GB]
Sector Sizes:     512 bytes logical, 4096 bytes physical
Rotation Rate:    Solid State Device
Form Factor:      2.5 inches
Device is:        In smartctl database [for details use: -P show]
ATA Version is:   ACS-2 T13/2015-D revision 3
SATA Version is:  SATA 2.6, 6.0 Gb/s (current: 6.0 Gb/s)
Local Time is:    Tue Apr 28 07:49:06 2020 CDT
SMART support is: Available - device has SMART capability.
SMART support is: Enabled
```


ceph config:
```
    osd_op_num_shards = 8
    osd_op_num_threads_per_shard = 2
    enable_throttle = false
    enable_codel = false
    enable_batch_bound = false
    kv_queue_upper_bound_size = 30
    bluestore_throttle_bytes = 100000000
    bluestore_throttle_deferred_bytes = 100000000
    bluestore_throttle_cost_per_io = 0
    bluestore_throttle_cost_per_io_hdd = 100000
    bluestore_throttle_cost_per_io_ssd = 100000
    bdev_block_size = 4096
    bluestore_min_alloc_size = 0
    bluestore_min_alloc_size_hdd = 65536
    bluestore_min_alloc_size_ssd =  4096
    bluestore_max_alloc_size = 0
    bluestore_prefer_deferred_size = 0
    bluestore_prefer_deferred_size_hdd = 0
    bluestore_prefer_deferred_size_ssd = 0
``` 
simple writes dominate

workload:
```
[global]
ioengine=rbd
clientname=admin
pool=mybench
rbdname=image1
rw=randwrite
bs=4096
;size=83886080000
time_based=1
runtime=300
direct = 1
;write_iops_log

[fio-write]
iodepth=48

```

Start Ceph cluster:\
`$sudo MON=1 OSD=1 MDS=0 ../src/vstart.sh -n -o 'bluestore block = /dev/sdc' -o 'bluestore block path = /dev/sdc' -o 'bluestore fsck on mkfs = false' -o 'bluestore fsck on mount = false' -o 'bluestore fsck on umount = false' -o 'bluestore block db path = ' -o 'bluestore block wal path = ' -o 'bluestore block wal create = false' -o 'bluestore block db create = false' --without-dashboard`

OSD is mapped to a raw device(without file system)

rbd info:

In [1]:
f = open('dump_rbd_info.txt', 'r')
file_contents = f.read()
print (file_contents)
f.close()

rbd image 'image1':
	size 1 GiB in 256 objects
	order 22 (4 MiB objects)
	snapshot_count: 0
	id: 1021d9fedd0
	block_name_prefix: rbd_data.1021d9fedd0
	format: 2
	features: layering, exclusive-lock, object-map, fast-diff, deep-flatten
	op_features: 
	flags: 
	create_timestamp: Tue May  5 01:30:02 2020
	access_timestamp: Tue May  5 01:30:02 2020
	modify_timestamp: Tue May  5 01:45:22 2020

